# generating figure 1
Tim Tyree<br>
9.22.2021

In [10]:
#conda activate pyenv_ub
from lib import *
import cuml
import cupy
import cudf
import tensorflow
import xgboost as xgb
import bokeh
import plotly
import seaborn as sns

In [21]:
darkmode=False
if darkmode:
    # For darkmode plots
    from jupyterthemes import jtplot
    jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [27]:
#TODO: load a csv file with annihilating trajectories from the Luo-Rudy model
input_fn_full='/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.025_Vthresh_-40/trajectories/ic001.21_traj_sr_600_mem_0.csv'
#TODO: load a csv file with annihilating trajectories from the particle model
input_fn_particle='/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_2_L_10_kappa_1500_varkappa_5/Log/pbc_particle_log1_log.csv'
df_full=cudf.read_csv(input_fn_full)
df_particle=cudf.read_csv(input_fn_particle)
df=df_full.copy()
df.head()

In [ ]:
#TODO: compute annihilation events for a particle from the Luo-Rudy model
kwargs={
    'input_fn_lst':file_name_list,
    'L':width,
    'DS':ds/width,
    'use_cache_0':True,
    'use_cache_1':True,
    'npartitions':npartitions,
    'sr':3*width,
    'mem':0,
    'trial_folder_name':trial_folder_name,
    'ic_suite_fn':ic_suite_fn
}
DS,width,trial_folder_name

In [29]:
input_fn=input_fn_full
ds=5
width=200
pid_col='particle'
t_col='t'
height=width
df=pd.read_csv(input_fn);print(input_fn)
DT=np.around(get_DT(df,pid_col=pid_col),5);print(f"the time resolution is {DT} ms.")
DS=ds/width
trial_folder_name=os.path.dirname(os.path.dirname(input_fn))

/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.025_Vthresh_-40/trajectories/ic001.21_traj_sr_600_mem_0.csv
the time resolution is 0.025 ms.


In [30]:
#print summary stats on particle lifetimes for one input folder
dft=df.groupby(pid_col)[t_col].describe()
df_lifetimes=-dft[['max','min']].T.diff().loc['min']
print(f"termination time was {df[t_col].max():.2f} ms")

print(f"printing summary stats on particle lifetimes for one input folder in {trial_folder_name}:")
print(df_lifetimes.describe())
# print(df_lifetimes.head(10))
print("\nPlease make a manual decision about minimum_lifetime, crop_start_by, and crop_end_by")

termination time was 6802.80 ms
printing summary stats on particle lifetimes for one input folder in /home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.025_Vthresh_-40:
count    82798.000000
mean         1.146551
std          2.425658
min         -0.000000
25%          0.175000
50%          0.375000
75%          0.900000
max         71.500000
Name: min, dtype: float64

Please make a manual decision about minimum_lifetime, crop_start_by, and crop_end_by


In [31]:
# #particle model
min_duration=40  #20. #ms # a pair of particles is considered if their lifetime is at least min_range
min_range   =0  #cm  # a pair of particles is considered if their max distance is at least min_range
filter_beginning=True
use_min_duration=True
use_grad_voltage=False
range_threshold=10 #cm #filter any deaths occurring at ranges exceeding range_threshold
round_t_to_n_digits=5
tmin=0.  #filter all times earlier than tmin? i think so?
max_dur=0.#ms

# #full model fk
# min_duration=300##20. #ms
# min_range   =.5  #cm
# filter_beginning=True
# use_min_duration=True
# use_grad_voltage=True
# range_threshold=.1 #cm #filter any deaths occurring at ranges exceeding range_threshold
# tmin=100#ms
# max_dur=150#ms #consider no more than the last max_dur milliseconds of either trajectory

#full model lr (and fk)
min_duration=50##20. #ms
min_range   =.5#1.#.2#  #cm #.2 and 1. had 3 events available...
filter_beginning=True #i think this does nothing...
use_min_duration=False
use_grad_voltage=True
range_threshold=.1 #cm #filter any deaths occurring at ranges exceeding range_threshold
tmin=100#ms
max_dur=149#500#ms#150# #consider no more than the last max_dur milliseconds of either trajectory


# filter_beginning=False
# use_min_duration=False

npartitions=os.cpu_count()-2
# npartitions=10#1

kwargs={
    'width':width,
    'height':height,
    'ds':ds,
#     'DT':DT,
    'printing':False,
    'min_range':min_range,
    'min_duration':min_duration,
    'npartitions':npartitions,
    'filter_beginning':filter_beginning,
    'use_grad_voltage':use_grad_voltage,
    'use_min_duration':use_min_duration,
    'range_threshold':range_threshold,
    'round_t_to_n_digits':round_t_to_n_digits,
    'tmin':tmin,
    'pid_col':pid_col,
    't_col':t_col,
    'max_dur':max_dur,
    'folder_out_name':f'annihilations_mindur_{min_duration}_maxdur_{max_dur}_minrange_{min_range}_rangethresh_{range_threshold}'
}#'folder_out_name':f'annihilations
kwargs

{'width': 200,
 'height': 200,
 'ds': 5,
 'printing': False,
 'min_range': 0.5,
 'min_duration': 50,
 'npartitions': 10,
 'filter_beginning': True,
 'use_grad_voltage': True,
 'use_min_duration': False,
 'range_threshold': 0.1,
 'round_t_to_n_digits': 5,
 'tmin': 100,
 'pid_col': 'particle',
 't_col': 't',
 'max_dur': 149,
 'folder_out_name': 'annihilations_mindur_50_maxdur_149_minrange_0.5_rangethresh_0.1'}

In [ ]:
#TODO: compute annihilation events for a particle from the particle model that is


In [ ]:
#TODO: select a particle from the Luo-Rudy model that is 
# - near the center of the domain, 
# - is long lived, and 
# - has a partner that is reasonably long-lived

In [ ]:
#TODO: compute annihilation events for a particle from the particle model that is
# - is long lived, and 
# - has a partner that is reasonably long-lived


In [22]:
#TODO: visualize an appealing spiral tip trajectory
# - swap out a few trajectories from the Luo-Rudy model and see what is most visually appealing

#TODO: translate scale and rotate the particle annihilation event onto that of the Luo-Rudy model
# - annihilation points must match
# - at least one initial point must match
#TODO: rotate and scale the other particle onto the other start point of the Luo-Rudy model
#TODO: visualize Fig. 1B
# - swap out a few trajectories from the Luo-Rudy model and see what is most visually appealing
# - swap out a few trajectories from the particle model and see what is most visually appealing
#TODO: scale and rotate the other start points to match

#TODO: load the initial conditions texture used by this particle

#TODO: generate a snapshot from the Luo-Rudy model
